# Analytics Notebook

### Connect To MongoDB

In [1]:
## Import Libraries
import os
from pymongo import MongoClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../.env.local')
db_uri = os.getenv('MONGODB_URI')

## Connect To DB
client = MongoClient(db_uri)
db = client['test']

In [2]:
# Testing
print(list(db['vehicles'].find()))

[{'_id': ObjectId('66ed791326d23c81b7e6c223'), 'licencePlate': '123', 'gpsApi': 'abc', 'make': 'Toyota', 'model': 'Camry', 'status': 'Draft', 'createdAt': datetime.datetime(2024, 9, 20, 13, 30, 59, 449000), 'updatedAt': datetime.datetime(2024, 9, 20, 13, 30, 59, 449000)}]


## Analytics Dashboard

In [3]:
## General Libraries
import json
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.holtwinters import ExponentialSmoothing

### Job Analytics

#### Job Type Distribution

In [4]:
# General Labels
job_types = ['ELECTRICIAN', 'VENTILATION', 'PLUMBER', 'HANDYMAN', 'AIRCON']

In [5]:
# Past 1 Month
total_job_frequency = [21, 6, 3, 19, 11]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-one-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)

average_job_frequency = total_job_frequency

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-one-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)   

In [6]:
# Past 3 Month
total_job_frequency = [56, 17, 10, 61, 29]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-three-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)  

average_job_frequency = [np.ceil(x / 3) for x in total_job_frequency]

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-three-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4) 

In [7]:
# Past 6 Month
total_job_frequency = [112, 33, 19, 106, 55]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-six-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)

average_job_frequency = [np.ceil(x / 6) for x in total_job_frequency]

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-six-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4) 

In [8]:
# Past 12 Month
total_job_frequency = [222, 61, 40, 205, 99]

data_job_type_dist = {
    'labels': job_types,
    'data': total_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-twelve-month-total.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4)

average_job_frequency = [np.ceil(x / 12) for x in total_job_frequency]

data_job_type_dist = {
    'labels': job_types,
    'data': average_job_frequency
}

# Save data to JSON file
with open('../public/analytics/job-type-dist-twelve-month-average.json', 'w') as file:
    json.dump(data_job_type_dist, file, indent=4) 

#### Job Duration Distribution

In [9]:
# Past 1 Month
job_duration = [21, 48, 57, 19, 181]

data_job_duration_dist = {
    'labels': job_types,
    'data': job_duration
}

# Save data to JSON file
with open('../public/analytics/job-duration-dist-one-month.json', 'w') as file:
    json.dump(data_job_duration_dist, file, indent=4) 

In [10]:
# Past 3 Month
job_duration = [20, 45, 52, 17, 166]

data_job_duration_dist = {
    'labels': job_types,
    'data': job_duration
}

# Save data to JSON file
with open('../public/analytics/job-duration-dist-three-month.json', 'w') as file:
    json.dump(data_job_duration_dist, file, indent=4) 

In [11]:
# Past 6 Month
job_duration = [23, 49, 58, 21, 174]

data_job_duration_dist = {
    'labels': job_types,
    'data': job_duration
}

# Save data to JSON file
with open('../public/analytics/job-duration-dist-six-month.json', 'w') as file:
    json.dump(data_job_duration_dist, file, indent=4) 

In [12]:
# Past 12 Month
job_duration = [29, 55, 62, 24, 191]

data_job_duration_dist = {
    'labels': job_types,
    'data': job_duration
}

# Save data to JSON file
with open('../public/analytics/job-duration-dist-twelve-month.json', 'w') as file:
    json.dump(data_job_duration_dist, file, indent=4) 

#### Job Revenue Distribution

In [13]:
# Past 1 Month
total_job_revenue = [800, 600, 600, 850, 2200]

data_job_revenue_dist = {
    'labels': job_types,
    'data': total_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-one-month-total.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)

average_job_revenue = total_job_revenue

data_job_revenue_dist = {
    'labels': job_types,
    'data': average_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-one-month-average.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)  

In [14]:
# Past 3 Month
total_job_revenue = [2490, 1710, 1500, 2400, 5130]

data_job_revenue_dist = {
    'labels': job_types,
    'data': total_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-three-month-total.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)

average_job_revenue = [round(x / 3, 2) for x in total_job_revenue]

data_job_revenue_dist = {
    'labels': job_types,
    'data': average_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-three-month-average.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)  

In [15]:
# Past 6 Month
total_job_revenue = [4490, 2970, 2700, 4400, 9330]

data_job_revenue_dist = {
    'labels': job_types,
    'data': total_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-six-month-total.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)

average_job_revenue = [round(x / 6, 2) for x in total_job_revenue]

data_job_revenue_dist = {
    'labels': job_types,
    'data': average_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-six-month-average.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)  

In [16]:
# Past 12 Month
total_job_revenue = [8270, 5130, 4860, 8130, 15570]

data_job_revenue_dist = {
    'labels': job_types,
    'data': total_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-twelve-month-total.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)

average_job_revenue = [round(x / 12, 2) for x in total_job_revenue]

data_job_revenue_dist = {
    'labels': job_types,
    'data': average_job_revenue
}

# Save data to JSON file
with open('../public/analytics/job-revenue-dist-twelve-month-average.json', 'w') as file:
    json.dump(data_job_revenue_dist, file, indent=4)  

#### Job Delay Distribution

In [17]:
# Past 1 Month
job_delay = [0, 0, 5, 0, 25]

data_job_delay_dist = {
    'labels': job_types,
    'data': job_delay
}

# Save data to JSON file
with open('../public/analytics/job-delay-dist-one-month.json', 'w') as file:
    json.dump(data_job_delay_dist, file, indent=4) 

In [18]:
# Past 3 Month
job_delay = [1, 2, 10,5, 40]

data_job_delay_dist = {
    'labels': job_types,
    'data': job_delay
}

# Save data to JSON file
with open('../public/analytics/job-delay-dist-three-month.json', 'w') as file:
    json.dump(data_job_delay_dist, file, indent=4) 

In [19]:
# Past 6 Month
job_delay = [4, 6, 12, 7, 50]

data_job_delay_dist = {
    'labels': job_types,
    'data': job_delay
}

# Save data to JSON file
with open('../public/analytics/job-delay-dist-six-month.json', 'w') as file:
    json.dump(data_job_delay_dist, file, indent=4) 

In [20]:
# Past 12 Month
job_delay = [8, 7, 15, 10, 60]

data_job_delay_dist = {
    'labels': job_types,
    'data': job_delay
}

# Save data to JSON file
with open('../public/analytics/job-delay-dist-twelve-month.json', 'w') as file:
    json.dump(data_job_delay_dist, file, indent=4) 

#### Job Revenue Forecasting

In [21]:
# Calculate the date 36 months ago from today
today = datetime.today()
start_date = today - pd.DateOffset(months=35)

def forecast_revenue(df, seasonal_periods, forecast_horizon):
    model = ExponentialSmoothing(df['revenue'], seasonal='add', seasonal_periods=seasonal_periods)
    model_fit = model.fit()
    forecast = model_fit.forecast(forecast_horizon)
    return forecast

In [22]:
# Electrician
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'revenue': [400, 310, 300, 320, 410, 450, 420, 350, 410, 520, 370, 350, 
                630, 630, 640, 550, 700, 710, 620, 580, 640, 750, 700, 680,
                800, 740, 690, 770, 1000, 900, 730, 850, 930, 1040, 700, 800]
}
electrician_df = pd.DataFrame(historical_data)
electrician_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = electrician_df['revenue'][-12:]
monthly_forecast = np.ceil(forecast_revenue(electrician_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-electrician-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = electrician_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['revenue'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_revenue(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-electrician-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [23]:
# Ventilation
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'revenue': [320, 260, 310, 390, 280, 260, 300, 230, 220, 240, 310, 340,  
                460, 430, 480, 560, 520, 510, 470, 470, 480, 410, 520, 530, 
                550, 640, 700, 780, 520, 600, 600, 550, 520, 580, 750, 670]
}
ventilation_df = pd.DataFrame(historical_data)
ventilation_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = ventilation_df['revenue'][-12:]
monthly_forecast = np.ceil(forecast_revenue(ventilation_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-ventilation-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = ventilation_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['revenue'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_revenue(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-ventilation-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [24]:
# Plumber
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'revenue': [280, 260, 300, 230, 320, 260, 310, 390, 220, 240, 310, 340,  
                520, 510, 470, 470, 460, 430, 480, 560, 480, 410, 520, 530, 
                520, 600, 600, 550, 550, 640, 700, 780, 520, 580, 750, 670]
}
plumber_df = pd.DataFrame(historical_data)
plumber_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = plumber_df['revenue'][-12:]
monthly_forecast = np.ceil(forecast_revenue(plumber_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-plumber-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = plumber_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['revenue'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_revenue(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-plumber-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [25]:
# Handyman
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'revenue': [420, 350, 410, 520, 370, 350, 400, 310, 300, 320, 410, 450,  
                620, 580, 640, 750, 700, 680, 630, 630, 640, 550, 700, 710, 
                730, 850, 930, 1040, 700, 800, 800, 740, 690, 770, 1000, 900]
}
handyman_df = pd.DataFrame(historical_data)
handyman_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = handyman_df['revenue'][-12:]
monthly_forecast = np.ceil(forecast_revenue(handyman_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-handyman-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = handyman_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['revenue'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_revenue(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-handyman-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [26]:
# Aircon
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'revenue': [840, 700, 600, 640, 820, 900, 820, 1040, 740, 700, 800, 620, 
                1240, 1160, 1280, 1100, 1400, 1420, 1280, 1500, 1400, 1360, 1260, 1260, 
                1460, 1700, 1380, 1540, 2000, 1800, 1860, 2080, 1400, 1600, 1600, 1480]
}
aircon_df = pd.DataFrame(historical_data)
aircon_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = aircon_df['revenue'][-12:]
monthly_forecast = np.ceil(forecast_revenue(aircon_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-aircon-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = aircon_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['revenue'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_revenue(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-revenue-forecast-aircon-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


#### Job Demand Forecast

In [27]:
def forecast_demand(df, seasonal_periods, forecast_horizon):
    model = ExponentialSmoothing(df['demand'], seasonal='add', seasonal_periods=seasonal_periods)
    model_fit = model.fit()
    forecast = model_fit.forecast(forecast_horizon)
    return forecast

In [28]:
# Electrician
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'demand': [1, 1, 0, 2, 4, 5, 2, 3, 4, 2, 3, 5, 
               3, 3, 4, 5, 7, 7, 2, 5, 6, 6, 7, 8,
               5, 4, 5, 6, 10, 9, 3, 5, 3, 4, 6, 8]
}
electrician_df = pd.DataFrame(historical_data)
electrician_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = electrician_df['demand'][-12:]
monthly_forecast = np.ceil(forecast_demand(electrician_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-electrician-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = electrician_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['demand'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_demand(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-electrician-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 


C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [29]:
# Ventilation
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'demand': [1, 0, 2, 4, 5, 3, 3, 4, 5, 6, 2, 1, 
               4, 10, 6, 5, 4, 3, 5, 2, 4, 3, 5, 1,
               8, 6, 4, 3, 5, 2, 7, 7, 5, 6, 2, 5]
}
ventilation_df = pd.DataFrame(historical_data)
ventilation_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = ventilation_df['demand'][-12:]
monthly_forecast = np.ceil(forecast_demand(ventilation_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-ventilation-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = ventilation_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['demand'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_demand(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-ventilation-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [30]:
# Plumber
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'demand': [10, 4, 6, 8, 2, 2, 0, 4, 8, 4, 6, 10, 
               14, 4, 10, 12, 6, 6, 8, 10, 14, 12, 14, 16,
               18, 6, 10, 6, 10, 8, 10, 12, 20, 8, 12, 16]
}
plumber_df = pd.DataFrame(historical_data)
plumber_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = plumber_df['demand'][-12:]
monthly_forecast = np.ceil(forecast_demand(plumber_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-plumber-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = plumber_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['demand'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_demand(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-plumber-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [31]:
# Handyman
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'demand': [7, 3, 4, 6, 2, 2, 0, 3, 6, 3, 4, 7, 
               9, 3, 7, 8, 4, 4, 6, 7, 9, 8, 9, 11,
               12, 4, 7, 4, 7, 6, 7, 8, 14, 6, 8, 11]
}
handyman_df = pd.DataFrame(historical_data)
handyman_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = handyman_df['demand'][-12:]
monthly_forecast = np.ceil(forecast_demand(handyman_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-handyman-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = handyman_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['demand'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_demand(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-handyman-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)


In [32]:
# Aircon
historical_data = {
    'date': pd.date_range(start=start_date, end=today, freq='M'),
    'demand': [47, 43, 44, 46, 42, 42, 40, 43, 36, 43, 44, 47, 
               59, 43, 47, 48, 44, 44, 46, 47, 49, 48, 52, 53,
               72, 49, 57, 48, 52, 55, 59, 58, 64, 60, 68, 71]
}
aircon_df = pd.DataFrame(historical_data)
aircon_df.set_index('date', inplace=True)

forecast_months = 6

# Monthly
historical_demand = aircon_df['demand'][-12:]
monthly_forecast = np.ceil(forecast_demand(aircon_df, seasonal_periods=12, forecast_horizon=forecast_months)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, monthly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(monthly_forecast.index[i].strftime("%Y-%m")) for i in range(len(monthly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_monthly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-aircon-month.json', 'w') as file:
    json.dump(data_job_demand_forecast_monthly, file, indent=4) 

# Quarterly 
df_quarterly = aircon_df.resample('Q').sum()
historical_demand = np.ceil(df_quarterly['demand'][-6:]).astype(int)
quarterly_forecast = np.ceil(forecast_demand(df_quarterly, seasonal_periods=4, forecast_horizon=forecast_months // 3)).astype(int)
combined_demand_forecast = np.concatenate((historical_demand, quarterly_forecast))

historical_labels = [historical_demand.index[i].strftime("%Y-%m") for i in range(len(historical_demand))]
forecasted_labels = [(quarterly_forecast.index[i].strftime("%Y-%m")) for i in range(len(quarterly_forecast))]
combined_labels = historical_labels + forecasted_labels

data_job_demand_forecast_quarterly = {
    'labels': combined_labels,
    'data': combined_demand_forecast.tolist()
}

with open('../public/analytics/job-demand-forecast-aircon-quarter.json', 'w') as file:
    json.dump(data_job_demand_forecast_quarterly, file, indent=4) 

C:\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
